In [3]:
%pip install -r local_req.txt

In [2]:
import torch
import torchaudio
import json

In [3]:
def pre_process(audio_path):
    waveform, sr = torchaudio.load(audio_path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
    return waveform

In [4]:
def get_label(label_pred):
    if type(label_pred) == list:
        label_pred = label_pred[0]
    if 'json_dict' not in globals():
        global json_dict
        with open("labels.json", "r") as f:
            json_dict = json.load(f)
    label = json_dict[str(label_pred)]
    return label

In [5]:
from BEATs import BEATs, BEATsConfig

def predict(audio_path, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    cfg = BEATsConfig(checkpoint['cfg'])
    BEATs_model = BEATs(cfg)
    BEATs_model.load_state_dict(checkpoint['model'])
    BEATs_model.eval()

    waveform = pre_process(audio_path)
    probs = BEATs_model.extract_features(waveform, padding_mask=None)[0]
    return probs

In [6]:
probs = predict('baby.wav', 'model.pt')
label_pred = probs.topk(k=1)[1].tolist()[0][0]
label = get_label(label_pred)
print(probs.size())
print(label_pred)
print(label)

torch.Size([1, 527])
431
Wail, moan
